In [ ]:
# Required Libraries
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from pdfminer.high_level import extract_text
from io import StringIO
import fitz
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os
import re
import torch
import requests
import json

In [ ]:
from datasets import load_dataset

# Load the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [ ]:
from transformers import T5Tokenizer

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Tokenize the data
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=150, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
from transformers import T5Tokenizer

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Tokenize the data
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=150, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
from pdfminer.high_level import extract_text_to_fp
from io import StringIO

def extract_text_from_pdf(pdf_path, start_page, end_page):
    text = ""
    output = StringIO()
    with open(pdf_path, 'rb') as f:
        extract_text_to_fp(f, output, page_numbers=range(start_page-1, end_page))
        text = output.getvalue()
    return text

# Define the path to the PDF and the pages to extract
pdf_path = "looma_sample_book.pdf"  # Update this to the path of your PDF file
start_page = 1  # Update this to the start page of Chapter 1
end_page = 2  # Update this to the end page of Chapter 1

# Extract text from the specified pages
chapter_text = extract_text_from_pdf(pdf_path, start_page, end_page)
print("Extracted Chapter Text:")
print(chapter_text)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
model_path = "./results/t5_fine_tuned"  # Path to the fine-tuned model
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Function to generate a summary
def generate_summary(chapter_text):
    inputs = tokenizer("summarize: " + chapter_text, return_tensors='pt', max_length=512, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Generate and print the summary for the extracted chapter text
summary = generate_summary(chapter_text)
print("Summary:")
print(summary)


In [18]:
import fitz  # PyMuPDF
import pytesseract

# Load PDF
pdf_path = "Math_76_Paper.pdf"  # Replace with the path to your PDF
pdf_document = fitz.open(pdf_path)

def extract_text_from_pdf(pdf_document, start_page, end_page):
    text_by_page = []
    for page_num in range(start_page - 1, end_page):
        page = pdf_document.load_page(page_num)
        text = page.get_text()
        if not text.strip():  # If no text found, use OCR
            images = convert_from_path(pdf_path, first_page=page_num+1, last_page=page_num+1)
            text = pytesseract.image_to_string(images[0])
        text_by_page.append(text)
    return text_by_page

# Extract text
chapter_1_text = extract_text_from_pdf(pdf_document, 0, 1)
chapter_1_text = "\n".join(chapter_1_text)
print(chapter_1_text)


Fig. 23.
Microsoft Stock Price Prediction with Sentiment
Fig. 24.
Microsoft Stock Price Prediction without Sentiment
• Without sentiment, Test MAE: 3.86
For Facebook, the inclusion of sentiment scores notably
enhanced the model’s forecasting accuracy, as indicated
by the lower RMSE and MAE values. This improvement
underscores the relevance of social media sentiment in
predicting Facebook’s stock price movements, highlighting
the importance of public perception and sentiment trends
in shaping the company’s market valuation.
Fig. 25.
Facebook Stock Price Prediction with Sentiment
Fig. 26.
Facebook Stock Price Prediction without Sentiment
13

Analyzing impact of twitter sentiment on stock market dynamics using
spectral clustering and deep learning
Vagmin Viswanathan, Rahul Gupta
Math 76.01: Topics in Applied Math
Department of Mathematics, Dartmouth College
Abstract— Modern social media platforms have become
influential forums for discussing and disseminating public
opinion
on
topics
like

In [19]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# model_name = "facebook/bart-large-cnn"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [21]:
# def generate_summary(text, max_length=150, num_beams=5):
#     inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
#     summary_ids = model.generate(inputs.input_ids, max_length=max_length, num_beams=num_beams, early_stopping=True)
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary

# # Generate summary for Chapter 1
# chapter_1_summary = generate_summary(chapter_1_text[:1024])  # Truncate text to 1024 tokens if too long

# print(f"Summary of Chapter 1:\n{chapter_1_summary}")

Summary of Chapter 1:
For Facebook, the inclusion of sentiment scores notablyenhanced the model’s forecasting accuracy, as indicated by the lower RMSE and MAE values. Modern social media platforms have becomeinfluential forums for discussing and disseminating publicopinion. This study highlights the importance of public perception and sentiment trends in shaping the company's market valuation.


In [13]:
# import requests
# import json

# url = "http://localhost:11434/api/chat"  # Ensure this is the correct URL for your local Llama3 instance

# def llama3(prompt):
#     data = {
#         "model": "llama3",
#         "messages": [
#             {
#                 "role": "user",
#                 "content": prompt
#             }
#         ],
#         "stream": False
#     }
    
#     headers = {
#         'Content-Type': 'application/json'
#     }
    
#     response = requests.post(url, headers=headers, json=data)
#     return response.json()['message']['content']


In [14]:
# # Generate summary for Chapter 1
# chapter_1_summary = llama3(f"Summarize the following text: {chapter_1_text}")

# print(f"Summary of Chapter 1:\n{chapter_1_summary}")


Summary of Chapter 1:
The text describes a study on predicting Microsoft stock prices using Twitter sentiments. The researchers built upon previous studies that showed the impact of investor sentiment on financial markets and introduced a novel approach that uses network analysis techniques to understand the structure of correlations between Twitter sentiments and stock prices.

The study collected Twitter data related to financial markets during the COVID-19 pandemic, which led to high market volatility and significant social media activity. The researchers used an existing dataset of around a million finance-related tweets from April 9, 2020, to July 16, 2020, as well as other datasets.

To create a sentiment time series, the researchers cleaned the Twitter data by removing duplicates, spam, and non-English tweets, and tagged each tweet with the company mentioned. They then extracted the date of each tweet and created a distribution of tweets across companies, showing that most tweet

In [5]:
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pytesseract

# Define a function to extract text from a page using PyMuPDF
def extract_text_from_page(page):
    text = page.get_text()
    if text.strip():  # If text is found, return it
        return text
    else:  # Otherwise, use OCR
        return None

# Define a function to perform OCR on an image of a page
def perform_ocr(image):
    return pytesseract.image_to_string(image)

# Define a function to extract text from a PDF, handling both text and scanned pages
def extract_text_from_pdf(pdf_path, start_page, end_page):
    pdf_document = fitz.open(pdf_path)
    text_by_page = []

    for page_num in range(start_page - 1, end_page):
        page = pdf_document.load_page(page_num)
        text = extract_text_from_page(page)
        if text is None:
            # If no text is found, convert the page to an image and perform OCR
            images = convert_from_path(pdf_path, first_page=page_num+1, last_page=page_num+1)
            text = perform_ocr(images[0])
        text_by_page.append(text)
    return text_by_page

# Example usage for Chapter 1 (from page 1 to 14)
pdf_path = "looma_sample_book.pdf"  # Replace with the path to your PDF
chapter_1_text = extract_text_from_pdf(pdf_path, 5, 14)
chapter_1_text = "\n".join(chapter_1_text)

# Display the extracted text
print(chapter_1_text)

—— Scientific Study

The scientific study includes scientific facts, theories,
models, experiments and physical situations. It studies various
objects in nature in a planned and organized way. It also logically
investigates the various phenomena based on the
scientific method and empirical or measurable evidence. Various
causative factors have a role in different events and phenomena that
have occurred in nature. The causative factor, characteristic, or trait
that has a direct or indirect relationship with an event or a phenom-
enonis known as a variable.

1.1 Variables of scientific research

When we see a phenomenon or change, we are curious to know
why it happens, what factors areinvolved, and how they have
affected that event or phenomenon. Such curiosity helps us to
predict the result of an action correctly. For instance, if we see
a wilted potted plant in the garden, we want to know why is it
wilted.Is it because of lack of water, overexposure to bright sun,
or someother reasons?

In [6]:
import requests
import json

url = "http://localhost:11434/api/chat"  # Ensure this is the correct URL for your local Llama3 instance

def llama3(prompt):
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False
    }
    
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.post(url, headers=headers, json=data)
    return response.json()['message']['content']

# Generate summary for Chapter 1
chapter_1_summary = llama3(f"Summarize the following text: {chapter_1_text}")

print(f"Summary of Chapter 1:\n{chapter_1_summary}")

with open("chapter_1_summary.txt", "w") as file:
    file.write(chapter_1_summary)

print("Summary saved to chapter_1_summary.txt")

Summary of Chapter 1:
The text discusses scientific units of measurement, specifically fundamental units and derived units. Fundamental units are those that have independent existence and cannot be resolved into simpler forms, such as the kilogram (mass), second (time), and meter (length). There are seven fundamental units in the SI system.

Derived units, on the other hand, are composed of two or more fundamental units and do not exist independently. Examples include density (kg/m³) and force (N = kg·m/s²).

The text also explains how derived units can be analyzed to determine their composition, using examples such as area (m²), force (N), and pressure (Pa). It also discusses the concept of unit analysis, where one checks that the units on both sides of an equation are the same. This is demonstrated through the example equation s = v·x·t, which is shown to be valid.

Finally, the text provides a table for students to classify various units used in daily life into fundamental and deriv